In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../../Sentiment_analysis/Primary_classification/Ternary_classification_2022.csv", encoding='UTF-8')
print(df.shape, df.columns)
df

(262308, 3) Index(['sentence', 'predicted_sentiment', 'org_idx'], dtype='object')


,sentence,predicted_sentiment,org_idx
0,치유 나무숲 야외 연구회 산림치유지도사 1급 산림치유 미래 유망자격 산림치유지도사 ...,긍정,0
1,치유 나무숲 회원님들 중에서도 이번 시험에 응시하시는 몇 분이 계십니다,부정,0
2,회원님들 응원과 격려를 위해 그리고 치유 나무숲 야외 연구회 홍보 그리고 1급 기출...,긍정,0
3,2024년 2월 평가시험에 응시하시는 수험생 및 관련 선생님들은 잠시 치유 나무숲 ...,긍정,0
4,출처 https license or kr board select noticeboa...,부정,0
...,...,...,...
262303,"예를 들어 현재 전북 전남 경남 3개 도에 걸쳐 있는 지리산 1,915m 부근을 비...",긍정,9974
262304,지리산이 걸쳐 있는 8개 행정구역 가운데 진주를 제외하면 모두 인구 소멸 우려지역이...,부정,9974
262305,지역을 비워두는 것도 한 전략이라는 말이다. 그리되면 캘리포니아 시에라네바다산맥 너...,긍정,9974
262306,문제는 내 지역구 소멸을 반대하는 지역 정치인들과 토호 세력들인데 국민 저변의 사회...,부정,9974


In [6]:
for emo in ['긍정','부정','중립']:

    ''' 형태소 분석 '''
    nouns = []
    verbs = []
    adjs = []

    df_class = df[df['predicted_sentiment'] == "%s"%emo]
    corpus = df_class['sentence'].to_list()

    for sent in tqdm(range(len(corpus))):
        # 문장에서 형태소/품사 추출
        try:
            a = okt.pos(corpus[sent], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
            for x, y in a:
                # 품사가 명사면 명사 리스트에 단어 추가
                if y == 'Noun':
                    nouns.append(x)
                # 품사가 동사면 동사 리스트에 단어 추가
                elif y == 'Verb':
                    verbs.append(x)
                # 품사가 형용사면 형용사 리스트에 단어 추가
                elif y == 'Adjective':
                    adjs.append(x)
        except:
            pass

    morph_dic = {'명사':nouns, '동사':verbs, '형용사':adjs}
    for mor in ['명사','동사','형용사']:
        
        # 명사 리스트에서 명사 빈도 리스트 생성
        morph_cnt = Counter(morph_dic[mor]).most_common()

        morph = []
        morph_freq = []
        for key, val in morph_cnt:
            morph.append(key)
            morph_freq.append(val)

        # 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
        morph_df = pd.DataFrame({'morph':morph, 'morph_freq':morph_freq})
        morph_df.to_csv("2022_감정예측_%s_%s.csv"%(emo, mor), index=False, encoding='UTF-8')

100%|██████████| 167/167 [00:00<00:00, 305.50it/s]


In [7]:
import plotly.express as px

# unique 값들의 개수를 계산
sentiment_counts = df['predicted_sentiment'].value_counts()


# 막대그래프로 시각화
fig = px.bar(sentiment_counts, x=sentiment_counts.index, y=sentiment_counts.values,
             labels={'x': 'Sentiment', 'y': 'Count'},
             title='Sentiment Distribution 2022',
             color=sentiment_counts.index,
             text=sentiment_counts.values)  # 개수를 막대에 표시하기 위해 text 속성에 sentiment_counts.values 할당

# 개수를 막대 위에 표시하는 옵션 추가
fig.update_traces(texttemplate='%{text}', textposition='outside')

# y축 범위 설정 (기존 최대값의 10% 더 큰 값으로 설정)
y_axis_range = sentiment_counts.max() * 1.1
fig.update_layout(yaxis_range=[0, y_axis_range])

# y축 값의 형식 설정 (정수 형식으로 표시)
fig.update_layout(yaxis_tickformat=',d')

# 글꼴 크기 설정
fig.update_layout(
    font=dict(
        family="Arial, bold",
        size=13
    )
)

# x축 레이블 설정
fig.update_xaxes(title_text='Label')

# 제목을 가운데에 위치시키기
fig.update_layout(title_x=0.5)

# 그래프 출력
fig.show()